# Search Tester Analysis
*Follow these steps to compare legacy results to the new api results.*

### Setup the environment

In [1]:
%run ./local/setup.ipynb

Unable to configure logging, attempted conf:/home/notebook/.local/lib/python3.10/site-packages/ppr_api/logging.conf


### Find search batches (choose the data set)
*find the ones you want to analyze*

In [2]:
search_type = None
after_date = None
before_date = None
### OPTIONAL: refine your batch search with any of the following
# search_type = SearchRequest.SearchTypes.XXXX
# before_date = _datetime_in_utc_
after_date = "2022-01-20 23:48:30.936128"
batches = TestSearchBatch.find_search_batches(search_type, after_date, before_date)
for batch in batches:
    print(batch.id, batch.test_date, batch.search_type, batch.sim_val_business, batch.sim_val_first_name, batch.sim_val_last_name)


781 2022-01-20 23:48:31.135637 BS 0.6 0.4 0.29
783 2022-01-21 00:02:26.450030 IS 0.6 0.4 0.29
784 2022-01-21 01:08:19.679522 SS 0.6 0.4 0.29


In [3]:
# add the ids of the batches you want to analyze
batch_ids = []
for batch in batches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. batch.id > _ , batch.sim_val_business > _ ...
        batch_ids.append(batch.id)

### Get the batch json

In [4]:
batches_to_analyze = []
for batch_id in batch_ids:
    batches_to_analyze.append(TestSearchBatch.find_by_id(batch_id))

# Analysis

### Exact Matches

#### Pass rate

In [5]:
searches_total = 0
searches_passed = 0
failed_searches = []
all_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        all_searches.append(search)
        if len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  0.9227386934673367
number of failed searches:  123


#### Failed Searches

*select failed searches to analyze*

In [6]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesExact']['resultsLegacy']), 'missed: ', len(json['matchesExact']['missedMatches']))

###########################################################
criteria:  HUMMERS CONTRACTING LTD                                               
total expected:  17 missed:  17
###########################################################
criteria:  J.ROSE CORPORATION                                                    
total expected:  2 missed:  2
###########################################################
criteria:  BC QUALITY HOMES LTD.                                                 
total expected:  1 missed:  1
###########################################################
criteria:  A.T. REALTY LTD                                                       
total expected:  1 missed:  1
###########################################################
criteria:  R.L.C. ENTERPRIZE LTD.                                                
total expected:  17 missed:  1
###########################################################
criteria:  DUNKIN BRANDS CANADA ULC                                          

###########################################################
criteria:  GREEN                    ANGELA         MARIE                         
total expected:  1 missed:  1
###########################################################
criteria:  KHAN                     MOHAMMED       IRSHAD                        
total expected:  11 missed:  8
###########################################################
criteria:  KAUR                     SANDEEP                                      
total expected:  38 missed:  1
###########################################################
criteria:  CAMPBELL                 DANIELLE                                     
total expected:  5 missed:  1
###########################################################
criteria:  MADAHAR                  RANJIT         K                             
total expected:  1 missed:  1
###########################################################
criteria:  KAUR                     SANDEEP                                   

###########################################################
criteria:  BROWN                    STEVEN         THEDORE                       
total expected:  2 missed:  2
###########################################################
criteria:  TODD                     RICHARD        ALLEN                         
total expected:  4 missed:  4
###########################################################
criteria:  BROWN                    STEVEN         THEODORE                      
total expected:  2 missed:  2
###########################################################
criteria:  SANGHERA                 MANDEEP        S                             
total expected:  5 missed:  3
###########################################################
criteria:  GODIN                    RICHARD        DANIEL                        
total expected:  2 missed:  2
###########################################################
criteria:  DHALIWAL                 HOSHIAR                                     

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [7]:
exact_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > _
        exact_searches_analysis.append(search)

*missed matches*

In [8]:
import re
### manually iterate through exact_searches_analysis one by one to see the missed matches
# search = exact_searches_analysis[1]
# for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])
### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

##################################################################################
criteria:  HUMMERS CONTRACTING LTD                                               
Missed Matches
-------------------------------------------------------------------------------
result: HUMMER'S CONTRACTING LTD                                       
reg num: 770314G
index:  0
-------------------------------------------------------------------------------
result: HUMMER'S CONTRACTING LTD                                       
reg num: 511320K
index:  1
-------------------------------------------------------------------------------
result: HUMMER'S CONTRACTING LTD                                       
reg num: 987566K
index:  2
-------------------------------------------------------------------------------
result: HUMMER'S CONTRACTING LTD                                       
reg num: 345623L
index:  3
-------------------------------------------------------------------------------
result: HUMMER'S CONTRAC

-------------------------------------------------------------------------------
result: BROWN                    C                              86DEC03
reg num: 098301M
index:  0
##################################################################################
criteria:  CHEN                     CHIA           SHU                           
Missed Matches
-------------------------------------------------------------------------------
result: CHEN                     CHIA                           44JUL24
reg num: 491367L
index:  0
-------------------------------------------------------------------------------
result: CHEN                     CHIA                           66OCT19
reg num: 290427L
index:  1
##################################################################################
criteria:  CHEN                     CHIA           SHU                           
Missed Matches
-------------------------------------------------------------------------------
result: CHEN           

-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         40DEC09
reg num: 388743E
index:  0
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         50JUN28
reg num: 877208K
index:  1
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         50OCT02
reg num: 158456L
index:  2
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         51FEB10
reg num: 872963L
index:  3
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         51FEB10
reg num: 376839M
index:  4
-------------------------------------------------------------------------------
result: BROWN            

-------------------------------------------------------------------------------
result: LEE                      CINDY                          54AUG17
reg num: 796522L
index:  0
-------------------------------------------------------------------------------
result: LEE                      CINDY                          78OCT31
reg num: 293068M
index:  1
-------------------------------------------------------------------------------
result: LEE                      CINDY                          83DEC18
reg num: 183846L
index:  2
-------------------------------------------------------------------------------
result: LEE                      CINDY                          90JAN08
reg num: 149552M
index:  3
-------------------------------------------------------------------------------
result: LEE                      CINDY                          90MAR15
reg num: 326480M
index:  4
-------------------------------------------------------------------------------
result: LEE              

-------------------------------------------------------------------------------
result: BROWN                    STEWART                               
reg num: 372193K
index:  0
##################################################################################
criteria:  VANDERMEER               CHRISTOPHER    JOHN                          
Missed Matches
-------------------------------------------------------------------------------
result: VANDERMEER               CHRISTOPHER                    72FEB10
reg num: 299517K
index:  0
##################################################################################
criteria:  CHEN                     BENSON                                       
Missed Matches
-------------------------------------------------------------------------------
result: MCMULLEN                 DAVID          J               79DEC18
reg num: 336967M
index:  1
-------------------------------------------------------------------------------
result: MCMULLEN       

-------------------------------------------------------------------------------
result: LEBLANC                  JAMIE                          79AUG15
reg num: 061960L
index:  0
##################################################################################
criteria:  LEBLANC                  JAMIE          ROGER                         
Missed Matches
-------------------------------------------------------------------------------
result: LEBLANC                  JAMIE                          79AUG15
reg num: 061960L
index:  0
##################################################################################
criteria:  SINGH                    AMNEET                                       
Missed Matches
-------------------------------------------------------------------------------
result: SINGH                    A                              91AUG21
reg num: 088703M
index:  0
##################################################################################
criteria:  SINGH    

-------------------------------------------------------------------------------
result: GILL                     GURPREET                              
reg num: 055492M
index:  0
-------------------------------------------------------------------------------
result: GILL                     GURPREET                       57FEB16
reg num: 329345M
index:  1
-------------------------------------------------------------------------------
result: GILL                     GURPREET                       75JUN29
reg num: 510710M
index:  2
-------------------------------------------------------------------------------
result: GILL                     GURPREET                       81MAR18
reg num: 528392L
index:  5
-------------------------------------------------------------------------------
result: GILL                     GURPREET                       81MAR22
reg num: 671724K
index:  6
-------------------------------------------------------------------------------
result: GILL             

-------------------------------------------------------------------------------
result: MACGREGOR                LESLIE                         67MAR08
reg num: 419884L
index:  0
-------------------------------------------------------------------------------
result: RIGGS                    DAVID          ERRON           69OCT21
reg num: 061667L
index:  1
##################################################################################
criteria:  CHADWICK                 STEVEN         MARK                          
Missed Matches
-------------------------------------------------------------------------------
result: CHADWICK                 STEVEN         M               74JUN27
reg num: 076888M
index:  0
##################################################################################
criteria:  BROWN                    STEVEN         THEODORE                      
Missed Matches
-------------------------------------------------------------------------------
result: BROWN          

-------------------------------------------------------------------------------
result: TR NHAF4625                  2013 ALTA-FAB 14 X 60-S-DIN       
reg num: 672740K
index:  0
##################################################################################
criteria:  1FTFW1EF8EFC03217                                                     
Missed Matches
-------------------------------------------------------------------------------
result: MV 5XYPHDA36KG498130         2019 KIA SORENTO                  
reg num: 977556L
index:  0
##################################################################################
criteria:  50-252                                                                
Missed Matches
-------------------------------------------------------------------------------
result: MV 50-252                    1994 ATECH LIBRA-02               
reg num: 574178M
index:  0
##################################################################################
criteria:  K6J2-6202

*results diff*

In [9]:
import re
### manually iterate through exact_searches_analysis one by one
# search = exact_searches_analysis[1]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('-------------------------------------------------------------')
    print('  legacy                           api')
    print('-------------------------------------------------------------')
    legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
    api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
    length = max(len(legacy_results), len(api_results))
    for i in range(length):
        if i < len(legacy_results) and i < len(api_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
        elif i < len(legacy_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
        elif i < len(api_results):
            print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')


##################################################################################
criteria:  HUMMERS CONTRACTING LTD                                               
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 770314G HUMMER'S CONTRACTING LTD  |
1: 511320K HUMMER'S CONTRACTING LTD  |
2: 987566K HUMMER'S CONTRACTING LTD  |
3: 345623L HUMMER'S CONTRACTING LTD  |
4: 531118L HUMMER'S CONTRACTING LTD  |
5: 754870L HUMMER'S CONTRACTING LTD  |
6: 713560J HUMMER'S CONTRACTING LTD.  |
7: 178439K HUMMER'S CONTRACTING LTD.  |
8: 236545K HUMMER'S CONTRACTING LTD.  |
9: 877030K HUMMER'S CONTRACTING LTD.  |
10: 017399L HUMMER'S CONTRACTING LTD.  |
11: 183989L HUMMER'S CONTRACTING LTD.  |
12: 238579L HUMMER'S CONTRACTING LTD.  |
13: 839503L HUMMER'S CONTRACTING LTD.  |
14: 917799L HUMMER'S CONTRACTING LTD.  |
15: 386228M HUMMER'S CONTRACTING LTD.  |
16: 466810M HUMMER'S CONTRACTING

0: 098301M BROWN C 86DEC03 | 956546K BROWN CHRISTINE L
1: 395220M BROWN CHRISTINE 67JUN15 | 973405M BROWN CHRISTINE LAURA
2: 993880K BROWN CHRISTINE A 47SEP20 | 956546K BROWN CHRISTINE LYN
3: 755848J BROWN CHRISTINE ADELE 59DEC29 | 847609L BROWN CHRISTINE ANN
4: 847609L BROWN CHRISTINE ANN 54JUL03 | 222814N BROWN CHRISTINE 
5: 956546K BROWN CHRISTINE L 52SEP22 | 222814N BROWN CHRISTINE JULIE
6: 460523H BROWN CHRISTINE LISE 52MAY07 | 993880K BROWN CHRISTINE A
7: 956546K BROWN CHRISTINE LYN 52SEP22 | 254360L BROWN CHRISTINE LYN
8: 254360L BROWN CHRISTINE LYN 52SEP22 | 395220M BROWN CHRISTINE 
9: 683702L BROWN CHRISTINE M 64MAR12 | 460523H BROWN CHRISTINE LISE
10:                    | 683702L BROWN CHRISTINE M
11:                    | 755848J BROWN CHRISTINE ADELE
##################################################################################
criteria:  CHEN                     CHIA           SHU                           
-------------------------------------------------------------
 

0: 633564L BROWN R WILLIAM 42FEB01 | 509644E BROWN RYAN GREGORY
1: 396437M BROWN RYAN 75JUN29 | 293023M BROWN RYAN 
2: 203999M BROWN RYAN 85NOV11 | 188413N BROWN RYAN WILLIAM
3: 293023M BROWN RYAN 85NOV11 | 124154N BROWN RYAN JONAS
4: 199086L BROWN RYAN 87DEC27 | 829928J BROWN RYAN WILLIAM
5: 375036M BROWN RYAN C  | 830343M BROWN RYAN R
6: 739705J BROWN RYAN DARCY 84JUN02 | 547370K BROWN RYAN MICHAEL
7: 976546K BROWN RYAN DAVID 66MAY04 | 188413N BROWN RYAN 
8: 388626K BROWN RYAN DOUGLAS 97FEB18 | 262462M BROWN RYAN LAWRENCE
9: 509547E BROWN RYAN GREGORY 74SEP19 | 830343M BROWN RYAN ROBERT
10: 509644E BROWN RYAN GREGORY 74SEP19 | 199086L BROWN RYAN 
11: 513862L BROWN RYAN J 82JAN10 | 709973M BROWN RYAN G
12: 372631K BROWN RYAN JAMES 82JAN10 | 164679N BROWN RYAN C
13: 372631K BROWN RYAN JAMES 82JAN10 | 356182H BROWN RYAN LESLIE
14: 513862L BROWN RYAN JAMES 82JAN10 | 739705J BROWN RYAN DARCY
15: 199086L BROWN RYAN JORDAN 87DEC27 | 164679N BROWN RYAN CURTIS
16: 262462M BROWN RYAN L 77FEB18

0: 149440L CHIA YEW 85MAR11 |
1: 149440L CHIA YEW-EN 85MAR11 |
2: 686986L CHIA YIM WE 70DEC25 |
3: 871272K CHIA YIM-FUN 62NOV02 |
4: 541764M CHIA YUN HUI 87SEP07 |
5: 541764M CHIA YUN-HUI I 87SEP07 |
##################################################################################
criteria:  BROWN                    STEWART        WOODROW                       
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 372193K BROWN STEWART  |
##################################################################################
criteria:  LEE                      JIMMY          SOO                           
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 749604J LEE JIMMY 79NOV15 |
##################################################################################
cr

0: 319779M CAMPBELL D A 45SEP06 | 474148K CAMPBELL DANIELLE PATRICE
1: 390367M CAMPBELL DANIELLE 79APR03 | 562130M CAMPBELL DANIELLE LEAN
2: 562130M CAMPBELL DANIELLE LEAN 73MAR23 | 390367M CAMPBELL DANIELLE PATRICE
3: 474148K CAMPBELL DANIELLE PATRICE 79APR03 | 390367M CAMPBELL DANIELLE 
4: 390367M CAMPBELL DANIELLE PATRICE 79APR03 |
##################################################################################
criteria:  MADAHAR                  RANJIT         K                             
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 505572K MADAHAR RANJIT KAUR 64SEP04 |
##################################################################################
criteria:  KAUR                     SANDEEP                                      
-------------------------------------------------------------
  legacy                           api
----------------------------

0: 733271B GEE MICHAEL 68JAN06 |
##################################################################################
criteria:  GILL                     JASKARAN       SINGH                         
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 338146E GILL JASKARAN  | 647825M GILL JASKARAN SINGH
1: 364582L GILL JASKARAN 96JUN01 | 658715L GILL JASKARAN SINGH
2: 125666L GILL JASKARAN S 93OCT26 | 442507L GILL JASKARAN SINGH
3: 658715L GILL JASKARAN S 98APR03 | 557473M GILL JASKARAN SINGH
4: 135910M GILL JASKARAN SINGH 68FEB19 | 125666L GILL JASKARAN SINGH
5: 557473M GILL JASKARAN SINGH 68FEB19 | 808919M GILL JASKARAN SINGH
6: 371836L GILL JASKARAN SINGH 73APR24 | 135910M GILL JASKARAN SINGH
7: 331593K GILL JASKARAN SINGH 90NOV24 | 331593K GILL JASKARAN SINGH
8: 442507L GILL JASKARAN SINGH 93JUL27 | 442507L GILL JASKARAN SINGH
9: 442507L GILL JASKARAN SINGH 93JUL27 | 3718

0: 208414L GILL AMARJIT 56MAR21 | 180359J GILL AMARJIT SINGH
1: 794982L GILL AMARJIT 56NOV03 | 712902I GILL AMARJIT SINGH
2: 261349M GILL AMARJIT 85MAY26 | 208414L GILL AMARJIT SINGH
3: 769985K GILL AMARJIT SINGH  | 127851M GILL AMARJIT SINGH
4: 712902I GILL AMARJIT SINGH 56JUN04 | 052286J GILL AMARJIT SINGH
5: 208414L GILL AMARJIT SINGH 56MAR21 | 776150L GILL AMARJIT SINGH
6: 754651L GILL AMARJIT SINGH 60JUN22 | 754651L GILL AMARJIT SINGH
7: 127851M GILL AMARJIT SINGH 60JUN22 | 727597M GILL AMARJIT SINGH
8: 052286J GILL AMARJIT SINGH 63AUG11 | 488563I GILL AMARJIT SINGH
9: 776150L GILL AMARJIT SINGH 63AUG11 | 363665L GILL AMARJIT SINGH
10: 180359J GILL AMARJIT SINGH 69MAY31 | 769985K GILL AMARJIT SINGH
11: 363665L GILL AMARJIT SINGH 69MAY31 | 006886N GILL AMARJIT SINGH
12: 488563I GILL AMARJIT SINGH 83OCT20 |
##################################################################################
criteria:  GILL                     GURPREET       KAUR                          
-------------

0: 126612M BROWN T A 66NOV19 | 647482L BROWN TED 
1: 647482L BROWN TED  | 647501L BROWN TED 
2: 647501L BROWN TED  | 920110M BROWN TED D
##################################################################################
criteria:  ANDERSON                 PATRICIA       LYNN                          
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 528343I ANDERSON PATRICIA 41APR13 | 699749L ANDERSON PATRICIA LYNN
1: 025759L ANDERSON PATRICIA 74JUL29 |
2: 296403M ANDERSON PATRICIA 86MAY21 |
3: 699749L ANDERSON PATRICIA LYNN 59JUN29 |
##################################################################################
criteria:  BROWN                    STEVEN                                       
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 894054K BROW

0: 177137I SANDHU JASWINDER 59OCT05 | 556544K SANDHU JASWINDER SINGH
1: 291875M SANDHU JASWINDER 80JUL13 | 291875M SANDHU JASWINDER SINGH
2: 364841L SANDHU JASWINDER 86JUN16 | 480510M SANDHU JASWINDER SINGH
3: 469492L SANDHU JASWINDER S 64AUG08 | 679447M SANDHU JASWINDER SINGH
4: 595775M SANDHU JASWINDER S 97FEB14 | 779166K SANDHU JASWINDER SINGH
5: 667804J SANDHU JASWINDER SINGH 72MAY30 | 595775M SANDHU JASWINDER SINGH
6: 480510M SANDHU JASWINDER SINGH 72MAY30 | 678373M SANDHU JASWINDER SINGH
7: 458535I SANDHU JASWINDER SINGH 75APR06 | 667804J SANDHU JASWINDER SINGH
8: 759124J SANDHU JASWINDER SINGH 80JUL13 | 759124J SANDHU JASWINDER SINGH
9: 779166K SANDHU JASWINDER SINGH 80JUL13 | 458535I SANDHU JASWINDER SINGH
10: 291875M SANDHU JASWINDER SINGH 80JUL13 | 917293M SANDHU JASWINDER SINGH
11: 005302M SANDHU JASWINDER SINGH 86JAN05 | 005302M SANDHU JASWINDER SINGH
12: 556544K SANDHU JASWINDER SINGH 88NOV05 |
13: 595775M SANDHU JASWINDER SINGH 97FEB14 |
##################################

### Similar Matches

#### Pass rate

In [5]:
searches_total = 0
searches_passed = 0
failed_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        # fails if missed matches
        if len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > 0:
            failed_searches.append(search)
        # fails if order is off
        elif search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value) != 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  0.5847989949748744
number of failed searches:  661


#### Failed searches

*select failed searches to analyze*

In [ ]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesSimilar']['resultsLegacy']), ' missed: ', len(json['matchesSimilar']['missedMatches']))
    print('first fail index: ', json['matchesSimilar']['firstFailIndex'])
    print('avg index diff: ', json['matchesSimilar']['avgIndexDiff'])

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [6]:
similar_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > _
        similar_searches_analysis.append(search)

*first failed indexes*

In [11]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

0


In [7]:
### avgs
total_no_fails = 0
total_fail_index = 0
for search in similar_searches_analysis:
    fail_index = search.fail_index(TestSearchResult.MatchType.SIMILAR.value)
    if fail_index == -1:
        total_no_fails += 1
    else:
        total_fail_index += fail_index

num_searches_failed = len(similar_searches_analysis) - total_no_fails
print('Number of searches with fail indexes: ', num_searches_failed)
print('avg fail index: ', total_fail_index/num_searches_failed)

Number of searches with fail indexes:  608
avg fail index:  0.32894736842105265


*avg order difference between legacy and api results (does NOT include missed matches)*

In [8]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

0


*missed matches*

In [9]:
import re
### manually iterate through
# search = similar_searches_analysis[0]
# print('criteria: ', search.search_criteria)
# print('Missed Matches')
# for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])

### print out all of them
for search in similar_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

##################################################################################
criteria:  SANGHA CONSULTING INC                                                 
Missed Matches
-------------------------------------------------------------------------------
result: SENGA CONSULTING INC.                                          
reg num: 953557G
index:  0
##################################################################################
criteria:  GRAPPA INVESTMENTS                                                    
Missed Matches
-------------------------------------------------------------------------------
result: GROWUP INVESTMENTS LTD                                         
reg num: 270351L
index:  0
-------------------------------------------------------------------------------
result: GROWUP INVESTMENTS LTD.                                        
reg num: 822944I
index:  1
-------------------------------------------------------------------------------
result: GROWUP INVESTME

-------------------------------------------------------------------------------
result: TWO PILLARS CONSTRUCTION (ALBERTA) LTD.                        
reg num: 467910H
index:  0
-------------------------------------------------------------------------------
result: TWO PILLARS CONSTRUCTION (ALBERTA) LTD.                        
reg num: 217812J
index:  1
-------------------------------------------------------------------------------
result: TWO PILLARS CONSTRUCTION (ALBERTA) LTD.                        
reg num: 836283J
index:  2
-------------------------------------------------------------------------------
result: TWO PILLARS CONSTRUCTION (ALBERTA) LTD.                        
reg num: 958224J
index:  3
-------------------------------------------------------------------------------
result: TWO PILLARS CONSTRUCTION LTD.                                  
reg num: 467896H
index:  4
-------------------------------------------------------------------------------
result: TWO PILLARS CONST

-------------------------------------------------------------------------------
result: VANSCAPE CONTRACTING LTD.                                      
reg num: 594309M
index:  0
-------------------------------------------------------------------------------
result: VANSCAPE LANDSCAPING                                           
reg num: 267009K
index:  1
-------------------------------------------------------------------------------
result: VANSCAPE LANDSCAPING                                           
reg num: 267009K
index:  2
-------------------------------------------------------------------------------
result: VANSCAPE LANDSCAPING AND CONTRACTING LTD.                      
reg num: 073274G
index:  3
-------------------------------------------------------------------------------
result: VANSCAPE LANDSCAPING AND CONTRACTING LTD.                      
reg num: 265478J
index:  4
-------------------------------------------------------------------------------
result: VANSCAPE LANDSCAP

##################################################################################
criteria:  MCAP FINANCIAL CORPORATION                                            
Missed Matches
##################################################################################
criteria:  LULU ISLAND WINERY LTD                                                
Missed Matches
##################################################################################
criteria:  LULU ISLAND WINERY LTD                                                
Missed Matches
##################################################################################
criteria:  SKEENA BIOENERGY LTD                                                  
Missed Matches
-------------------------------------------------------------------------------
result: SKEENA BIOENERGY CORPORATION                                   
reg num: 948953L
index:  0
##################################################################################
criteria:  THREE P

##################################################################################
criteria:  123DENTIST INC.                                                       
Missed Matches
-------------------------------------------------------------------------------
result: 123DENTIST CORPORATION                                         
reg num: 915294K
index:  0
-------------------------------------------------------------------------------
result: 123DENTIST CORPORATION                                         
reg num: 915030K
index:  1
##################################################################################
criteria:  123DENTIST INC.                                                       
Missed Matches
-------------------------------------------------------------------------------
result: 123DENTIST CORPORATION                                         
reg num: 915294K
index:  0
-------------------------------------------------------------------------------
result: 123DENTIST CORP

...

##################################################################################
criteria:  5AEAH1512CH000106                                                     
Missed Matches
##################################################################################
criteria:  5NPEC4AC0DH526988                                                     
Missed Matches
##################################################################################
criteria:  5AEAH1513RH000635                                                     
Missed Matches
##################################################################################
criteria:  5AEAH1515EH001138                                                     
Missed Matches
##################################################################################
criteria:  5AEAH1510EH000608                                                     
Missed Matches
##################################################################################
criteria:  1FMCU9

##################################################################################
criteria:  CATIT38GJJAN00313                                                     
Missed Matches
##################################################################################
criteria:  CAT00740CB1R00425                                                     
Missed Matches
##################################################################################
criteria:  1NPTX4EX9HD447493                                                     
Missed Matches
##################################################################################
criteria:  CAT00740TB1R00541                                                     
Missed Matches
##################################################################################
criteria:  5UXKR0C51F0P02387                                                     
Missed Matches
##################################################################################
criteria:  1FTEW1

-------------------------------------------------------------------------------
result: MV 1C6RR7MT4JS201049         2018 RAM 1500                     
reg num: 531594K
index:  1
-------------------------------------------------------------------------------
result: MV 2HGFC1E57LH201049         2020 HONDA CIVIC                  
reg num: 519906M
index:  2
-------------------------------------------------------------------------------
result: MV 3C63R3SLXLG201049         2020 RAM 3500 LARAMIE LTD C/C     
reg num: 602507M
index:  3
-------------------------------------------------------------------------------
result: MV 4YDT28B29EN201049         2014 KEYSTONE RV COMPANY HID      
reg num: 375400H
index:  4
-------------------------------------------------------------------------------
result: MV CAT00D8TAKPZ01049         2006 CATERPILLAR D8T              
reg num: 964422L
index:  5
##################################################################################
criteria:  7PZ00742   

##################################################################################
criteria:  CAT0330DCL2K00223                                                     
Missed Matches
##################################################################################
criteria:  CAT00D7RKAGN00633                                                     
Missed Matches
##################################################################################
criteria:  IJL97238G617                                                          
Missed Matches
##################################################################################
criteria:  SADCJ2BV1HA891361                                                     
Missed Matches
##################################################################################
criteria:  1FTEW1EG7HFC70321                                                     
Missed Matches
##################################################################################
criteria:  CAT001

##################################################################################
criteria:  2M9PL3037JP016357                                                     
Missed Matches
##################################################################################
criteria:  2M9PL3035JP016356                                                     
Missed Matches
##################################################################################
criteria:  2DETBDZ3181023751                                                     
Missed Matches
##################################################################################
criteria:  1FMJU1MT0GEF02891                                                     
Missed Matches
##################################################################################
criteria:  202C                                                                  
Missed Matches
-------------------------------------------------------------------------------
result: MV 0BNK00202

*results diff*

In [10]:
missed = 0
for search in similar_searches_analysis:
    if search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
        missed += 1

print('searches with missed matches: ', missed)
print('percent of total that have a missed similar: ', (searches_total-missed)/searches_total)

# import re
# ### manually iterate through similar_searches_analysis one by one
# search = similar_searches_analysis[0]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
# for search in similar_searches_analysis:
#     print('criteria: ', search.search_criteria)
#     print('-------------------------------------------------------------')
#     print('  legacy                           api')
#     print('-------------------------------------------------------------')
#     legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
#     api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
#     length = max(len(legacy_results), len(api_results))
#     for i in range(length):
#         if i < len(legacy_results) and i < len(api_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#         elif i < len(legacy_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#         elif i < len(api_results):
#             print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

searches with missed matches:  277
percent of total that have a missed similar:  0.8260050251256281


### Create MD file of notebook run
**NOTE:** save notebook (i.e. _cmd s_) now to have results show in markdown file

In [17]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [18]:
md_name = nb_name[:-6]+'.md'

In [ ]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2